<a href="https://colab.research.google.com/github/MasaHirai-Jinen/Medical_Institution_db_colab/blob/main/%E5%8C%BB%E7%99%82%E6%A9%9F%E9%96%A2%E5%8F%97%E7%90%86%E5%B1%8A%E9%A0%85%E7%9B%AE%E5%88%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# google 認証
google cloud storageに接続する。

In [ ]:
from google.cloud import storage
from google.colab import auth
auth.authenticate_user()

project_id = 'clean-mountain-258802'
bucket_name = 'jmidb-bucket'

client = storage.Client(project=project_id)
bucket = client.get_bucket(bucket_name)


# データ取得

In [ ]:
import pandas as pd
import io
import re
import tqdm

In [ ]:
%%time
blobs = bucket.list_blobs(prefix='accept_eath')

dfs = []
for blob in blobs:
    dfs.append(pd.read_excel(io.BytesIO(blob.download_as_bytes())))

CPU times: user 2min 4s, sys: 1.37 s, total: 2min 5s
Wall time: 4min 3s


## すべてのdataframeを一つにまとめる
以下のカラムを追加
- 「更新日」：いつ現在のデータか
- 「種別」：`医科`・`歯科`・`薬局`のどれか



In [ ]:
complete_df = pd.DataFrame()
for df in dfs:
  indexs = tuple(df[df.iloc[:,0].str.startswith('[', na=False)].index)
  new_df = pd.DataFrame()
  for i, index in enumerate(indexs):
    block_df = df.iloc[index:indexs[i+1]].copy() if i < len(indexs)-1 else df.iloc[index:].copy()

    string = block_df.iloc[0,0]
    match_ = re.search(r'\[(.*)\s*現在\s*(.*)\]', string)
    update_date, type_ = match_.groups()

    # 3列目をカラム名として使用
    block_df.columns = block_df.iloc[2]
    block_df = block_df.iloc[3:,:]

    block_df.loc[:, '更新日'] = update_date
    block_df.loc[:, '種別'] = type_

    new_df = pd.concat([new_df, block_df])

  # new_dfから"項番"の列がNaNの行を削除
  new_df.dropna(inplace=True, subset=['項番'])
  complete_df = pd.concat([complete_df, new_df])

complete_df.reset_index(drop=True, inplace=True)
print(complete_df.size)
complete_df.head()

<ipython-input-4-bbbecd94c9ba>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  block_df['更新日'] = update_date
<ipython-input-4-bbbecd94c9ba>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  block_df['種別'] = type_


6875106


,項番,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,...,FAX番号,病床数,受理記号,受理番号,算定開始年月日,個別有効開始年月日,備考（見出し）,備考（データ）,更新日,種別
0,1,31,鳥取県,情報通信機器を用いた診療に係る基準,0111097,0130028,NaN,鳥取県立中央病院,680－0901,鳥取市江津７３０,...,NaN,一般 504\n結核 10\n一般（感染）\n ...,情報通信,第12号,令和 4年 4月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科
1,2,31,鳥取県,情報通信機器を用いた診療に係る基準,0112418,0131927,NaN,鳥取市立病院,680－8501,鳥取市的場一丁目１番地,...,0857-37-1558,一般 340,情報通信,第14号,令和 4年 5月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科
2,3,31,鳥取県,情報通信機器を用いた診療に係る基準,0113085,NaN,NaN,加藤医院,689－1201,鳥取市用瀬町用瀬３８２,...,0858-87-2672,NaN,情報通信,第45号,令和 6年 5月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科
3,4,31,鳥取県,情報通信機器を用いた診療に係る基準,0113259,NaN,NaN,石田医院,689－0501,鳥取市青谷町青谷４０３２番地１９,...,NaN,NaN,情報通信,第3号,令和 4年 4月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科
4,5,31,鳥取県,情報通信機器を用いた診療に係る基準,0113333,NaN,NaN,医療法人社団 本城内科クリニック,680－0007,鳥取市湯所町２丁目１１０番地,...,NaN,NaN,情報通信,第17号,令和 4年 6月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科


In [ ]:
complete_df.to_excel('complete_df.xlsx', index=False)

In [ ]:
complete_df.tail()

,項番,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,...,FAX番号,病床数,受理記号,受理番号,算定開始年月日,個別有効開始年月日,備考（見出し）,備考（データ）,更新日,種別
327381,1,22,静岡県,新生児治療回復室入院医療管理料,0310146,0810067,NaN,順天堂大学医学部附属静岡病院,410－2295,伊豆の国市長岡１１２９番地,...,0559(48)5088,一般 633,新回復,第8号,平成30年 2月 1日,NaN,NaN,NaN,令和 6年 4月 1日,医科
327382,2,22,静岡県,新生児治療回復室入院医療管理料,4160042,4160048,NaN,静岡済生会総合病院,422－8527,静岡市駿河区小鹿一丁目１番１号,...,054(285)5179,一般 578,新回復,第5号,平成23年11月 1日,NaN,NaN,NaN,令和 6年 4月 1日,医科
327383,3,22,静岡県,新生児治療回復室入院医療管理料,4210789,4210199,NaN,静岡県立こども病院,420－8660,静岡市葵区漆山８６０番地,...,054(247)6259,一般 243\n精神 36,新回復,第10号,平成30年 9月 1日,NaN,NaN,NaN,令和 6年 4月 1日,医科
327384,4,22,静岡県,新生児治療回復室入院医療管理料,7110465,7110172,NaN,浜松医療センター,432－8580,浜松市中央区富塚町３２８番地,...,053(452)9217,一般 600\n一般（感染）\n 6,新回復,第3号,令和 6年 1月 1日,NaN,NaN,NaN,令和 6年 4月 1日,医科
327385,5,22,静岡県,新生児治療回復室入院医療管理料,9610488,9610484,NaN,浜松医科大学医学部附属病院,431－3192,浜松市中央区半田山一丁目２０番１号,...,053(435)2153,一般 576\n精神 37,新回復,第1号,令和 4年11月 1日,NaN,NaN,NaN,令和 6年 4月 1日,医科


In [ ]:
{'項番':int,
 '都道府県コード':int,
 '都道府県名':str,
 '受理届出名称':str,
 '医療機関番号':str,
 '併設医療機関番号':str,
 '医療機関記号番号':str,
 '医療機関名称':str,
 '医療機関所在地（郵便番号）':str,
 '医療機関所在地（住所）':str,
 '電話番号':str,
 'FAX番号':str,
 '病床数':str,
 '受理記号':str,
 '受理番号':str,
 '算定開始年月日':str,
 '個別有効開始年月日':str,
 '備考（見出し）':str,
 '備考（データ）':str,
 '更新日':str,
 '種別':str}

In [ ]:
# complete_dfの型変換
astype = {'項番':int,
 '都道府県コード':int,
 '都道府県名':str,
 '受理届出名称':str,
 '医療機関番号':str,
 '併設医療機関番号':str,
 '医療機関記号番号':str,
 '医療機関名称':str,
 '医療機関所在地（郵便番号）':str,
 '医療機関所在地（住所）':str,
 '電話番号':str,
 'FAX番号':str,
 '病床数':str,
 '受理記号':str,
 '受理番号':str,
 '算定開始年月日':str,
 '個別有効開始年月日':str,
 '備考（見出し）':str,
 '備考（データ）':str,
 '更新日':str,
 '種別':str}

complete_df.astype(astype).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327386 entries, 0 to 327385
Data columns (total 21 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   項番             327386 non-null  int64 
 1   都道府県コード        327386 non-null  int64 
 2   都道府県名          327386 non-null  object
 3   受理届出名称         327386 non-null  object
 4   医療機関番号         327386 non-null  object
 5   併設医療機関番号       327386 non-null  object
 6   医療機関記号番号       327386 non-null  object
 7   医療機関名称         327386 non-null  object
 8   医療機関所在地（郵便番号）  327386 non-null  object
 9   医療機関所在地（住所）    327386 non-null  object
 10  電話番号           327386 non-null  object
 11  FAX番号          327386 non-null  object
 12  病床数            327386 non-null  object
 13  受理記号           327386 non-null  object
 14  受理番号           327386 non-null  object
 15  算定開始年月日        327386 non-null  object
 16  個別有効開始年月日      327386 non-null  object
 17  備考（見出し）        327386 non-null  object
 18  備考（デ

# データクリーニング

In [ ]:
df = complete_df.copy()

In [ ]:
# 特定の`更新日`に絞り込む

target_update = '令和 6年 5月 1日'

df = df[df['更新日'].str.contains(target_update)]
df.head()

,項番,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,...,FAX番号,病床数,受理記号,受理番号,算定開始年月日,個別有効開始年月日,備考（見出し）,備考（データ）,更新日,種別
0,1,31,鳥取県,情報通信機器を用いた診療に係る基準,0111097,0130028,NaN,鳥取県立中央病院,680－0901,鳥取市江津７３０,...,NaN,一般 504\n結核 10\n一般（感染）\n ...,情報通信,第12号,令和 4年 4月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科
1,2,31,鳥取県,情報通信機器を用いた診療に係る基準,0112418,0131927,NaN,鳥取市立病院,680－8501,鳥取市的場一丁目１番地,...,0857-37-1558,一般 340,情報通信,第14号,令和 4年 5月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科
2,3,31,鳥取県,情報通信機器を用いた診療に係る基準,0113085,NaN,NaN,加藤医院,689－1201,鳥取市用瀬町用瀬３８２,...,0858-87-2672,NaN,情報通信,第45号,令和 6年 5月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科
3,4,31,鳥取県,情報通信機器を用いた診療に係る基準,0113259,NaN,NaN,石田医院,689－0501,鳥取市青谷町青谷４０３２番地１９,...,NaN,NaN,情報通信,第3号,令和 4年 4月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科
4,5,31,鳥取県,情報通信機器を用いた診療に係る基準,0113333,NaN,NaN,医療法人社団 本城内科クリニック,680－0007,鳥取市湯所町２丁目１１０番地,...,NaN,NaN,情報通信,第17号,令和 4年 6月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科


In [ ]:
# 特定の受理届出名称で絞り込む
target_name = '特定集中治療室管理料'

df = df[df['受理届出名称'].str.startswith(target_name)]
df.head()

,項番,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,...,FAX番号,病床数,受理記号,受理番号,算定開始年月日,個別有効開始年月日,備考（見出し）,備考（データ）,更新日,種別
41761,1,31,鳥取県,特定集中治療室管理料１,8010028,8030022,NaN,鳥取大学医学部附属病院,683－8504,米子市西町３６番地１,...,0859-38-7159,一般 649\n結核 6\n精神 40\n一...,集１,第5号,令和 4年10月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科
41762,1,31,鳥取県,特定集中治療室管理料１,8010028,8030022,NaN,鳥取大学医学部附属病院,683－8504,米子市西町３６番地１,...,0859-38-7159,一般 649\n結核 6\n精神 40\n一...,集１,第5号,令和 4年10月 1日,NaN,NaN,早期離床・リハビリテーション加算,令和 6年 5月 1日,医科
41763,1,31,鳥取県,特定集中治療室管理料１,8010028,8030022,NaN,鳥取大学医学部附属病院,683－8504,米子市西町３６番地１,...,0859-38-7159,一般 649\n結核 6\n精神 40\n一...,集１,第5号,令和 4年10月 1日,NaN,NaN,早期栄養介入管理加算,令和 6年 5月 1日,医科
41764,1,31,鳥取県,特定集中治療室管理料１,8010028,8030022,NaN,鳥取大学医学部附属病院,683－8504,米子市西町３６番地１,...,0859-38-7159,一般 649\n結核 6\n精神 40\n一...,集１,第5号,令和 4年10月 1日,NaN,NaN,算定上限日数に係る施設基準,令和 6年 5月 1日,医科
41765,1,31,鳥取県,特定集中治療室管理料１,8010028,8030022,NaN,鳥取大学医学部附属病院,683－8504,米子市西町３６番地１,...,0859-38-7159,一般 649\n結核 6\n精神 40\n一...,集１,第5号,令和 4年10月 1日,NaN,NaN,当該保険医療機関内に専任の小児科医が常時配置されている,令和 6年 5月 1日,医科


In [ ]:
# 備考から病床数を抽出
bed_num = df[df['備考（見出し）'].str.contains('病床数', na=False)]['備考（データ）'].str.extract(r'(\d+)').astype(int)

bed_df = df.loc[bed_num.index, ['都道府県コード', '医療機関番号']]
bed_df['病床数'] = bed_num
bed_df.drop_duplicates(inplace=True)

bed_df

,都道府県コード,医療機関番号,病床数
41767,31,8010028,6
41774,31,0111097,6
41783,31,8010028,18
41937,32,0113738,4
41942,32,0412015,12
...,...,...,...
323852,07,0411497,10
323854,07,0211772,12
323855,07,0211772,8
323858,07,0313586,10


In [ ]:
bed_df[bed_df.duplicated(keep=False,subset=['都道府県コード', '医療機関番号'])]

,都道府県コード,医療機関番号,病床数
41767,31,8010028,6
41783,31,8010028,18
42522,34,8010017,10
42549,34,8010017,6
82743,09,5210074,8
82750,09,5210074,12
82760,09,5210074,10
92382,11,6300297,12
92429,11,6300297,14
101416,12,0411736,12


In [ ]:
# dfとbed_dfを都道府県コード,医療機関番号で結合
## bed_dfにないデータ病床数を-1で補完
df = df.drop_duplicates(subset=['都道府県コード', '医療機関番号']).merge(bed_df, on=['都道府県コード', '医療機関番号'], how='left')
df['病床数_y'] = df['病床数_y'].fillna(-1)
df.rename(columns={'病床数_y':'病床数'}, inplace=True)
df['病床数'] = df['病床数'].astype(int)
df.head()

,項番,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,...,病床数_x,受理記号,受理番号,算定開始年月日,個別有効開始年月日,備考（見出し）,備考（データ）,更新日,種別,病床数
0,1,31,鳥取県,特定集中治療室管理料１,8010028,8030022,NaN,鳥取大学医学部附属病院,683－8504,米子市西町３６番地１,...,一般 649\n結核 6\n精神 40\n一...,集１,第5号,令和 4年10月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科,6.0
1,1,31,鳥取県,特定集中治療室管理料１,8010028,8030022,NaN,鳥取大学医学部附属病院,683－8504,米子市西町３６番地１,...,一般 649\n結核 6\n精神 40\n一...,集１,第5号,令和 4年10月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科,18.0
2,1,31,鳥取県,特定集中治療室管理料２,0111097,0130028,NaN,鳥取県立中央病院,680－0901,鳥取市江津７３０,...,一般 504\n結核 10\n一般（感染）\n ...,集２,第4号,令和 5年 5月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科,6.0
3,1,32,島根県,特定集中治療室管理料１,0113738,0131512,NaN,松江市立病院,690－8509,松江市乃白町３２番地１,...,一般 \n 一般 416\n精神 50\nその他 4,集１,第5号,令和 4年 5月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科,4.0
4,1,32,島根県,特定集中治療室管理料２,0412015,0430591,NaN,島根県立中央病院,693－8555,出雲市姫原四丁目１番地１,...,一般 522\n精神 40\n一般（感染）\n ...,集２,第24号,令和 6年 3月 1日,NaN,NaN,NaN,令和 6年 5月 1日,医科,12.0


In [ ]:
df = df[["都道府県コード",
    "都道府県名",
    "医療機関番号",
    "医療機関名称",
    "医療機関所在地（郵便番号）",
    "医療機関所在地（住所）",
    "電話番号",
    "受理届出名称",
    "受理番号",
    "病床数",
    "更新日"]]

df.sort_values(by=['都道府県コード', '医療機関番号'], inplace=True)
df

<ipython-input-71-319387af25c5>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by=['都道府県コード', '医療機関番号'], inplace=True)


,都道府県コード,都道府県名,医療機関番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,受理届出名称,受理番号,病床数,更新日
50,01,北海道,0113719,ＮＴＴ東日本 札幌病院,060－0061,札幌市中央区南１条西１５丁目２９０番地,011-623-7000,特定集中治療室管理料３,第219号,4,令和 6年 5月 1日
43,01,北海道,0116381,市立札幌病院,060－8604,札幌市中央区北１１条西１３丁目１番１号,011-726-2211,特定集中治療室管理料２,第30号,8,令和 6年 5月 1日
38,01,北海道,0119385,札幌医科大学附属病院,060－8543,札幌市中央区南１条西１６丁目２９１番地,011-611-2111,特定集中治療室管理料１,第39号,10,令和 6年 5月 1日
51,01,北海道,0213873,医療法人 徳洲会 札幌東徳洲会病院,065－0033,札幌市東区北３３条東１４丁目３－１,011-722-1110,特定集中治療室管理料３,第211号,10,令和 6年 5月 1日
52,01,北海道,0219771,札幌心臓血管クリニック,007－0849,札幌市東区北４９条東１６丁目８番１号,011-784-7847,特定集中治療室管理料３,第205号,8,令和 6年 5月 1日
...,...,...,...,...,...,...,...,...,...,...,...
482,47,沖縄県,8110028,沖縄県立中部病院,904－2243,うるま市宮里２８１番地,098(973)4111,特定集中治療室管理料２,第6号,14,令和 6年 5月 1日
488,47,沖縄県,8110150,沖縄県立北部病院,905－0017,名護市大中２－１２－３,0980-52-2719,特定集中治療室管理料３,第24号,4,令和 6年 5月 1日
483,47,沖縄県,8110176,沖縄県立南部医療センター・こども医療センター,901－1105,島尻郡南風原町新川１１８番地の１,098-888-0123,特定集中治療室管理料２,第2号,15,令和 6年 5月 1日
489,47,沖縄県,8110184,沖縄赤十字病院,902－0076,那覇市与儀１丁目３番１号,098(853)3134,特定集中治療室管理料３,第37号,6,令和 6年 5月 1日


In [ ]:
df.to_excel('clean_df.xlsx', index=False)

# 2024/10/08 新規作成

In [ ]:
import pandas as pd
import numpy as np
import io
import re
import tqdm
import pprint

In [ ]:
origin_df = pd.read_excel('/content/accept_eath_kinki_2024-10-03_2024.10_tokutei1.xlsx')
origin_df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,[令和 6年10月 1日 現在 医科] 令和６年１０月１日作成,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,項番,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,FAX番号,病床数,受理記号,受理番号,算定開始年月日,個別有効開始年月日,備考（見出し）,備考（データ）
3,1,18,福井県,救命救急入院料１,0117059,NaN,NaN,福井県立病院,910－8526,福井市四ツ井２－８－１,0776-54-5151,0776-57-2945,一般 551\n一般（感染）\n 4\n結核 ...,救１,第2号,令和 6年 4月 1日,NaN,NaN,NaN
4,1,18,福井県,救命救急入院料１,0117059,NaN,NaN,福井県立病院,910－8526,福井市四ツ井２－８－１,0776-54-5151,0776-57-2945,一般 551\n一般（感染）\n 4\n結核 ...,救１,第2号,令和 6年 4月 1日,NaN,NaN,早期離床・リハビリテーション加算
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,5,30,和歌山県,ハイケアユニット入院医療管理料１,8210124,8230128,和医1012,日本赤十字社 和歌山医療センター,640－8269,和歌山市小松原通４丁目２０番地,073-422-4171,073-426-1168,一般 692\n一般（感染）\n 8,ハイケア１,第14号,令和 5年 4月 1日,NaN,当該治療室の病床数:,12床
1569,6,30,和歌山県,ハイケアユニット入院医療管理料１,8410377,8430371,田医1037,独立行政法人国立病院機構南和歌山医療センター,646－0015,田辺市たきない町２７－１,0739-26-7050,0739-24-2055,一般 316,ハイケア１,第12号,令和 4年 8月 1日,NaN,NaN,NaN
1570,6,30,和歌山県,ハイケアユニット入院医療管理料１,8410377,8430371,田医1037,独立行政法人国立病院機構南和歌山医療センター,646－0015,田辺市たきない町２７－１,0739-26-7050,0739-24-2055,一般 316,ハイケア１,第12号,令和 4年 8月 1日,NaN,NaN,早期離床・リハビリテーション加算
1571,6,30,和歌山県,ハイケアユニット入院医療管理料１,8410377,8430371,田医1037,独立行政法人国立病院機構南和歌山医療センター,646－0015,田辺市たきない町２７－１,0739-26-7050,0739-24-2055,一般 316,ハイケア１,第12号,令和 4年 8月 1日,NaN,NaN,早期栄養介入管理加算


In [ ]:
from dataclasses import dataclass

@dataclass
class SyubetuType:
    id: int
    name: str

class SyubetuData:
    __df: pd.DataFrame = pd.DataFrame(
        [[1, "医科"],
        [3, "歯科"],
        [4, "調剤"],
        ],
        columns=["種別ID", "種別名"]
    )

    @property
    def df(self) -> pd.DataFrame:
        return self.__df

    def find_syubetu_from_id(self, id:int) -> SyubetuType|None:
        if id in self.__df["種別ID"].values:
            return SyubetuType(id=id, name=self.__df[self.__df["種別ID"] == id]["種別名"].values[0])
        return None

    def find_syubetu_from_name(self, name:str) -> SyubetuType|None:
        if name in self.__df["種別名"].values:
            return SyubetuType(id=self.__df[self.__df["種別名"] == name]["種別ID"].values[0], name=name)
        return None

    def create_table_sql(self, con):
        sql = """
        CREATE TABLE IF NOT EXISTS syubetu (
            id INT PRIMARY KEY,
            name VARCHAR(2)
        );
        """
        self.__df.to_sql('syubetu', con, if_exists='replace', index=False)

SYUBETU = SyubetuData()

print("1", SYUBETU.find_syubetu_from_id(1))
print("2", SYUBETU.find_syubetu_from_id(2))

1 SyubetuType(id=1, name='医科')
2 None


In [ ]:
df = origin_df.copy()
df.dropna(axis=0, how='all', inplace=True)

string = df.iloc[0,:]
string = string.str.cat(sep=' ')
match_ = re.search(r'\[(.*)\s*現在\s*(.*)\]', string)
update_date, type_ = match_.groups()
syubetu = SYUBETU.find_syubetu_from_name(type_)
if syubetu is None:
    raise ValueError(f"Warning: Syubetu not found for type: {type_}")

df.columns = df.iloc[1]
df = df[df['項番'].str.isnumeric()]
df.reset_index(drop=True, inplace=True)

df.loc[:, '更新日'] = update_date
df.loc[:, '種別ID'] = syubetu.id
df.loc[:, '医療機関コード'] = df['都道府県コード'].astype(str).str.zfill(2)\
                              + df['種別ID'].astype(str)\
                              + df['医療機関番号'].astype(str).str.zfill(7)

df

2,項番,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,...,病床数,受理記号,受理番号,算定開始年月日,個別有効開始年月日,備考（見出し）,備考（データ）,更新日,種別ID,医療機関コード
0,1,18,福井県,救命救急入院料１,0117059,NaN,NaN,福井県立病院,910－8526,福井市四ツ井２－８－１,...,一般 551\n一般（感染）\n 4\n結核 ...,救１,第2号,令和 6年 4月 1日,NaN,NaN,NaN,令和 6年10月 1日,1,1810117059
1,1,18,福井県,救命救急入院料１,0117059,NaN,NaN,福井県立病院,910－8526,福井市四ツ井２－８－１,...,一般 551\n一般（感染）\n 4\n結核 ...,救１,第2号,令和 6年 4月 1日,NaN,NaN,早期離床・リハビリテーション加算,令和 6年10月 1日,1,1810117059
2,1,18,福井県,救命救急入院料１,0117059,NaN,NaN,福井県立病院,910－8526,福井市四ツ井２－８－１,...,一般 551\n一般（感染）\n 4\n結核 ...,救１,第2号,令和 6年 4月 1日,NaN,当該治療室の病床数：,16床,令和 6年10月 1日,1,1810117059
3,1,18,福井県,救命救急入院料１,0117059,NaN,NaN,福井県立病院,910－8526,福井市四ツ井２－８－１,...,一般 551\n一般（感染）\n 4\n結核 ...,救１,第2号,令和 6年 4月 1日,NaN,救命救急センターに係る事項：,充実段階がＡである,令和 6年10月 1日,1,1810117059
4,1,18,福井県,救命救急入院料１,0117059,NaN,NaN,福井県立病院,910－8526,福井市四ツ井２－８－１,...,一般 551\n一般（感染）\n 4\n結核 ...,救１,第2号,令和 6年 4月 1日,NaN,NaN,精神疾患診断治療初回加算,令和 6年10月 1日,1,1810117059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541,5,30,和歌山県,ハイケアユニット入院医療管理料１,8210124,8230128,和医1012,日本赤十字社 和歌山医療センター,640－8269,和歌山市小松原通４丁目２０番地,...,一般 692\n一般（感染）\n 8,ハイケア１,第14号,令和 5年 4月 1日,NaN,当該治療室の病床数:,12床,令和 6年10月 1日,1,3018210124
1542,6,30,和歌山県,ハイケアユニット入院医療管理料１,8410377,8430371,田医1037,独立行政法人国立病院機構南和歌山医療センター,646－0015,田辺市たきない町２７－１,...,一般 316,ハイケア１,第12号,令和 4年 8月 1日,NaN,NaN,NaN,令和 6年10月 1日,1,3018410377
1543,6,30,和歌山県,ハイケアユニット入院医療管理料１,8410377,8430371,田医1037,独立行政法人国立病院機構南和歌山医療センター,646－0015,田辺市たきない町２７－１,...,一般 316,ハイケア１,第12号,令和 4年 8月 1日,NaN,NaN,早期離床・リハビリテーション加算,令和 6年10月 1日,1,3018410377
1544,6,30,和歌山県,ハイケアユニット入院医療管理料１,8410377,8430371,田医1037,独立行政法人国立病院機構南和歌山医療センター,646－0015,田辺市たきない町２７－１,...,一般 316,ハイケア１,第12号,令和 4年 8月 1日,NaN,NaN,早期栄養介入管理加算,令和 6年10月 1日,1,3018410377


In [ ]:
# ['都道府県コード','医療機関番号','受理届出名称']でグループ分け
grouped = df.groupby(['医療機関コード','受理届出名称'])
pprint.pprint(tuple(grouped.groups.keys()))

(('1810117059', 'ハイケアユニット入院医療管理料１'),
 ('1810117059', '救命救急入院料１'),
 ('1810117059', '特定集中治療室管理料６'),
 ('1810117117', '特定集中治療室管理料６'),
 ('1810117117', '脳卒中ケアユニット入院医療管理料'),
 ('1810117539', '特定集中治療室管理料５'),
 ('1810118909', 'ハイケアユニット入院医療管理料１'),
 ('1810118909', '脳卒中ケアユニット入院医療管理料'),
 ('1810214070', 'ハイケアユニット入院医療管理料１'),
 ('1810414068', '救命救急入院料１'),
 ('1818010058', 'ハイケアユニット入院医療管理料１'),
 ('1818010058', '特定集中治療室管理料６'),
 ('2510100155', '救命救急入院料３'),
 ('2510100155', '特定集中治療室管理料２'),
 ('2510107374', '特定集中治療室管理料１'),
 ('2510202043', '特定集中治療室管理料５'),
 ('2510202043', '脳卒中ケアユニット入院医療管理料'),
 ('2510300102', 'ハイケアユニット入院医療管理料１'),
 ('2510301373', '特定集中治療室管理料３'),
 ('2510301373', '脳卒中ケアユニット入院医療管理料'),
 ('2510401322', 'ハイケアユニット入院医療管理料１'),
 ('2510401322', '特定集中治療室管理料５'),
 ('2510601970', 'ハイケアユニット入院医療管理料１'),
 ('2510601970', '特定集中治療室管理料３'),
 ('2510700475', 'ハイケアユニット入院医療管理料１'),
 ('2510701101', 'ハイケアユニット入院医療管理料１'),
 ('2510701101', '特定集中治療室管理料５'),
 ('2511200194', '特定集中治療室管理料２'),
 ('2511200194', '脳卒中ケアユニット入院医療管理料'),
 ('25114022

In [ ]:
kikan = grouped.get_group(('3018012785', '特定集中治療室管理料２'))
kikan

2,項番,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,...,病床数,受理記号,受理番号,算定開始年月日,個別有効開始年月日,備考（見出し）,備考（データ）,更新日,種別ID,医療機関コード
1513,1,30,和歌山県,特定集中治療室管理料２,8012785,8032789,和医1278,和歌山県立医科大学附属病院,641－0012,和歌山市紀三井寺８１１番地１,...,一般 \n 一般 760\n精神 \n 精神 40,集２,第4号,令和 4年10月 1日,NaN,NaN,NaN,令和 6年10月 1日,1,3018012785
1514,1,30,和歌山県,特定集中治療室管理料２,8012785,8032789,和医1278,和歌山県立医科大学附属病院,641－0012,和歌山市紀三井寺８１１番地１,...,一般 \n 一般 760\n精神 \n 精神 40,集２,第4号,令和 4年10月 1日,NaN,NaN,当該保険医療機関内に広範囲熱傷特定集中治療を担当する常勤の医師が勤務している,令和 6年10月 1日,1,3018012785
1515,1,30,和歌山県,特定集中治療室管理料２,8012785,8032789,和医1278,和歌山県立医科大学附属病院,641－0012,和歌山市紀三井寺８１１番地１,...,一般 \n 一般 760\n精神 \n 精神 40,集２,第4号,令和 4年10月 1日,NaN,NaN,当該保険医療機関内に専任の小児科医が常時配置されている,令和 6年10月 1日,1,3018012785
1516,1,30,和歌山県,特定集中治療室管理料２,8012785,8032789,和医1278,和歌山県立医科大学附属病院,641－0012,和歌山市紀三井寺８１１番地１,...,一般 \n 一般 760\n精神 \n 精神 40,集２,第4号,令和 4年10月 1日,NaN,病床数：,10床,令和 6年10月 1日,1,3018012785


In [ ]:
bed_num = kikan[kikan['備考（見出し）'].str.contains('病床数', na=False)]
if bed_num.empty:
    print("no bed_num")
else:
    print(','.join(bed_num['備考（データ）']))

,備考（データ）
1516,10床


## クラス概要

1. `ExcelDataExtractor`
- 役割：Excelファイルの読み込みとデータ抽出に特化
- 理由：単一責任の原則に従い、Excel処理に関する責務のみを持つ
- 主な機能：データの読み取り、基本的なバリデーション

2. `DataTransformer`
- 役割：抽出したデータのDB格納用フォーマットへの変換
- 理由：データ変換ロジックを分離し、再利用性を高める
- 主な機能：データクレンジング、日付フォーマット変換など

3. `DatabaseHandler`
- 役割：データベース操作の一元管理
- 理由：
- 主な機能：接続管理、データの挿入・更新

4. `DataMigrationService`
- 役割：全体のワークフローを制御
- 理由：
- 主な機能：他のコンポーネントの連携、エラーハンドリング

**クラス図**

![クラス図](https://www.plantuml.com/plantuml/svg/TL7DQZCn3BpdANpCHyWBxE4Zq0PwwQcz2yNM9gPUEqZQD53wxbrtVmpknYSn6diyCmTHPEsxiB41HSpnRYaye-BnhenM4vklZHdEthG-45nHBuqHvLzmPsYaWKF5xRy9l67m0q05pVU12Hry9lwG2v4E-FTcKis2xunHsiGTpP8xepC6ci2T88-es_piygeDXB5I6daP96Wz1IUrsWc5NZ2wGCMWJJ6ILPyY36FhpurntkgVRMEUbddywBnS4d1UQiX78Hw3AWO6lBykKLN6cFAYwDOUNlsPCI_UY6_UqcA4vkwQbJudZfQucoh_cJ84VXaZQghCIljaUoNevf-K2iQRGCo9fS0XdGiRH57xgPjLH_lz_pKB3xcraSV4oi-1ekkxy0C0)

In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, List
import pandas as pd
import sqlalchemy as sa
from logging import Logger

In [ ]:
class BaseExtractor(ABC):
    file_path: str
    df: pd.DataFrame|None
    header_info: pd.DataFrame|None
    @abstractmethod
    def extract_data(self) -> pd.DataFrame:
        pass

    @abstractmethod
    def validate_data(self, df: pd.DataFrame) -> bool:
        pass

class ExcelDataExtractor(BaseExtractor):
    """
    Excelファイルの読み込みとデータ抽出に特化
    単一責任の原則に従い、Excel処理に関する責務のみを持つ
    """
    def __init__(self, file_path: str):
        self.file_path = file_path
        self.df = None
        self.header_info = None

    def read_excel(self):
        dfs = pd.read_excel(self.file_path, sheet_name=None,dtype=str)
        for k, df in dfs.item()
            df.dropna(axis=0, how='all', inplace=True)
            df.columns = df.iloc[1]

            header_info = self.__extract_header(df)
            if self.header_info is None:
                self.header_info = header_info
            else:
                self.header_info = pd.concat([self.header_info, header_info])

            df = df[df['項番'].str.isnumeric()]
            df.reset_index(drop=True, inplace=True)

            if self.df is None:
                self.df = df
            else:
                self.df = pd.concat([self.df, df])

    def __extract_header(self, df):
        df = df.copy()
        df = df[~df['項番'].str.isnumeric()]
        df = df.drop_duplicates()
        df = df[df['項番']=='項番']
        return df

In [ ]:
class BaseTransformer(ABC):
    @abstractmethod
    def transform(self) -> pd.DataFrame:
        """
        Excelファイルから抽出されたデータをDB格納用フォーマットに変換する。
        """
        pass

    @abstractmethod
    def __extract_header(self) -> datetime, str:
        """
        Excelファイルのヘッダー情報から更新日と種別を抽出する。
        """
        pass

    @abstractmethod
    def __bed_num(self) -> int:
        """
        備考から病床数を抽出する。
        """
        pass

class DataTransformer(BaseTransformer):
    def __init__(self, df: pd.DataFrame, header_info: pd.DataFrame):
        self.df = df
        self.header_info = header_info

    def transform(self) -> pd.DataFrame:
        # TODO:
        pass

    def __extract_header(self):
        # TODO:
        pass

    def __bed_num(self):
        # TODO:
        pass

In [ ]:
import sqlite3

class BaseDatabaseHandler(ABC):

    @abstractmethod
    def insert_data(self, data: pd.DataFrame):
        pass

    @abstractmethod
    def initialize_database(self):
        """データベースとテーブルを初期化する"""
        pass

class DatabaseHandler(BaseDatabaseHandler):
    tables = [
        "Institution",
        "FacilityStandard",
        "Main",
        "BedNum"
    ]

    def __init__(self, db_name:str):
        self.db_name = db_name

    # dbと接続するためのデコレーターを定義
    def connect(self, func):
        def wrapper(*args, **kwargs):
            self.connection = sqlite3.connect(self.db_name)
            yiled
            self.connection.close()
        return wrapper

    def disconnect(self):
        if self.connection:
            self.connection.close()

    def insert_data(self, data: pd.DataFrame):
        # TODO: データ挿入ロジック
        pass

    @self.connect
    def init_table(self):
        cursor = self.connection.cursor()
        # 全テーブルを削除
        for table in self.tables:
            cursor.execute(f"DROP TABLE IF EXISTS {table};")
        self.commit()

        sql_statements =( """
            -- 医療機関テーブル（Institution）
            CREATE TABLE Institution (
                code_id INT PRIMARY KEY AUTO_INCREMENT,
                医療機関コード VARCHAR(10),
                医療機関番号 INT,
                併設医療機関番号 INT,
                医療機関名称 VARCHAR(255),
                都道府県コード INT,
                都道府県名 VARCHAR(5),
                郵便番号 VARCHAR(20),
                住所 VARCHAR(255),
                電話番号 VARCHAR(20),
                FAX番号 VARCHAR(20)
            );
            """,
            """
            -- 施設基準テーブル（FacilityStandard）
            CREATE TABLE FacilityStandard (
                受理_id INT PRIMARY KEY AUTO_INCREMENT,
                受理届出名称 VARCHAR(255),
                受理記号 VARCHAR(50)
            );
            """,
            """
            -- メインテーブル（Main）
            CREATE TABLE Main (
                main_id INT PRIMARY KEY AUTO_INCREMENT,
                code_id VARCHAR(255),
                受理_id INT,
                受理番号 VARCHAR(255),
                算定開始年月日 VARCHAR(10),
                個別有効開始年月日 VARCHAR(10),
                更新日 DATETIME,
                UNIQUE (code_id, 受理_id),
                FOREIGN KEY (code_id) REFERENCES Institution(code_id),
                FOREIGN KEY (受理_id) REFERENCES FacilityStandard(受理_id)
            );
            """,
            """
            -- 抽出したベッド数テーブル（BedNum）
            CREATE TABLE BedNum (
                bed_id INT PRIMARY KEY AUTO_INCREMENT,
                main_key INT,
                病床数 INT,
                FOREIGN KEY (main_id) REFERENCES main(main_id)
            );
        """
        )
        for sql in sql_statements:
            cousor.execute(sql)
        self.commit()

In [ ]:
class BaseMigrationService(ABC):
    extractor: BaseExtractor
    transformer: BaseTransformer
    db_handler: BaseDatabaseHandler
    logger: Logger

    @abstractmethod
    def migrate(self):
        pass


class DataMigrationService(BaseMigrationService):
    def __init__(self,
                extractor: BaseExtractor
                transformer: BaseTransformer
                db_handler: BaseDatabaseHandler
                logger: Logger):
        self.extractor = extractor
        self.transformer = transformer
        self.db_handler = db_handler
        self.logger = logger

    def migrate(self):
        try:
            excel_data = self._extract_data()
            transformed_data = self._transform_data(excel_data)
            self._load_data()
        except Exception as error:
            self._handle_errors(error)

    def _extract_data(self):
        self.logger.info("Extracting data from Excel file...")
        excel_data = self.extractor.extract_data()
        self.logger.info("Data extraction completed.")
        return excel_data

    def _transform_data(self, excel_data:DataFrame):
        self.logger.info("Transforming data to DB format...")
        transformed_data = self.transformer.transform_to_db_format(self.data)
        self.logger.info("Data transformation completed.")
        return transformed_data

    def _load_data(self):
        self.logger.info("Loading data into the database...")
        self.db_handler.connect()
        self.db_handler.insert_data(self.data)
        self.db_handler.disconnect()

    def _handle_errors(self, error: Exception):
        self.logger.error(f"Migration failed: {str(error)}")
        raise

    def _log_migration_status(self, message: str):
        self.logger.info(message)

In [ ]:
DataMigrationService

**抽出するデータ列**
- "都道府県コード" # 重複
- "都道府県名" # 重複
- "受理届出名称"
- "医療機関番号" # 重複
- "併設医療機関番号" # 重複
- "医療機関記号番号" # 重複
- "医療機関名称" # 重複
- "受理記号"
- "受理番号"
- "算定開始年月日"
- "個別有効開始年月日"
- "備考（見出し）"
- "備考（データ）"

テーブル情報

- 機関テーブル
    - "医療機関コード": STR PRIMEKEY
    - "医療機関番号":INT
    - "併設医療機関番号": INT
    - "医療機関名称": STR
    - "都道府県コード": INT
    - "都道府県名": STR
    - "郵便番号": STR
    - "住所": STR
    - "電話番号": STR
    - "FAX番号": STR

- 施設基準テーブル
    - "受理コード":INT PRIMEKEY
    - "受理届出名称": STR
    - "受理記号": STR

- mainテーブル
    - "index":INT PRIMEKEY
    - "医療機関コード": STR
    - "受理コード":INT
    - "受理番号": STR
    - "算定開始年月日": STR
    - "個別有効開始年月日": STR
    - "病床数": STR
    - "開始日": DATETIME
    - "終了日": DATETIME

- mainテーブルの医療機関コードは重複可
- mainテーブルの受理コードは重複可
- mainテーブルの医療機関コードと受理コードの組み合わせの重複不可

![ER図](https://www.plantuml.com/plantuml/png/XLD1Yzf06BtdLmmvjRJv0sLPTikw4CfAsqsWjv9kSWXT8wpni6X13MqrMXIAIQq1eI1geQ9KASQc_J5n9kPVT1mrcY7GdCBttfltvitCMKwNxlHyvfPHD5tLxm6BHhtG-iO2AGTuBOTdULtDQg300Fmz0gB8Nu2RhAoyLcLmSlBysUafGQw5bm0sba6dibt0h_pqZT--KICednPYQD3-3fk_8cZbpTV37mc4f9LXwrCmc4JQqF32ycVeEe7JY9jYOGp5mgFvVFNdB-MoyfgeLZfmkilrQ4fnBizVxIRlc4DdbhV9ttEHQv7cBgKRzHP3rxgaoTAT7AyFDksWzI6fG0B0gGa_kjjjKj1w-8Molf9KZLXcy4_CXame4-u_vyIHIhTU4K9be0e9nZOSVmsjEXpKuMA6d2go-mT9gGwhVUJKeBb8uboS2oc1lqe1r9qXQx8VYsd-XPW23nAJFgO3FeoTXUdXzlooxPTxlj7n3GCpK7j2IdggoEby9bxJ6qMcMo8jaFhUAlVHJGlizz0ryL8Rk-EtKInoNBP01E09O1D3imnz9sWPjIcM8S2Ej0kEQVjOB7EcQFBcvJB_0000)

In [ ]:
class DatabaseInitializer:
    def __init__(self, db_handler: DatabaseHandler):
        self.db_handler = db_handler

    def initialize_database(self):
        self.db_handler.connect()
        self._delete_tables()
        self._create_tables()
        self.db_handler.disconnect()

    def _delete_tables(self):
        sql = """
            DROP TABLE IF EXISTS Institution;
            DROP TABLE IF EXISTS FacilityStandard;
            DROP TABLE IF EXISTS Main;
            DROP TABLE IF EXISTS BedNum;
        """
        self.db_handler.execute_query(sql)
        self.db_handler.commit()

    def _create_tables(self):
        # テーブルの作成ロジック
        sql_statements =( """
            -- 医療機関テーブル（Institution）
            CREATE TABLE Institution (
                code_id INT PRIMARY KEY AUTO_INCREMENT,
                医療機関コード VARCHAR(10),
                医療機関番号 INT,
                併設医療機関番号 INT,
                医療機関名称 VARCHAR(255),
                都道府県コード INT,
                都道府県名 VARCHAR(5),
                郵便番号 VARCHAR(20),
                住所 VARCHAR(255),
                電話番号 VARCHAR(20),
                FAX番号 VARCHAR(20)
            );
            """,
            """
            -- 施設基準テーブル（FacilityStandard）
            CREATE TABLE FacilityStandard (
                受理_id INT PRIMARY KEY AUTO_INCREMENT,
                受理届出名称 VARCHAR(255),
                受理記号 VARCHAR(50)
            );
            """,
            """
            -- メインテーブル（Main）
            CREATE TABLE Main (
                main_id INT PRIMARY KEY AUTO_INCREMENT,
                code_id VARCHAR(255),
                受理_id INT,
                受理番号 VARCHAR(255),
                算定開始年月日 VARCHAR(10),
                個別有効開始年月日 VARCHAR(10),
                更新日 DATETIME,
                UNIQUE (code_id, 受理_id),
                FOREIGN KEY (code_id) REFERENCES Institution(code_id),
                FOREIGN KEY (受理_id) REFERENCES FacilityStandard(受理_id)
            );
            """,
            """
            -- 抽出したベッド数テーブル（BedNum）
            CREATE TABLE BedNum (
                bed_id INT PRIMARY KEY AUTO_INCREMENT,
                main_key INT,
                病床数 INT,
                FOREIGN KEY (main_id) REFERENCES main(main_id)
            );
        """
        )
        for sql in sql_statements:
            self.db_handler.execute_query(sql)
        self.db_handler.commit()

In [ ]:
# 備考見出しから"病床数"の文字列を含む行を抽出
l_data = []
l_columns = [
    "医療機関コード",
    "医療機関番号",
    "併設医療機関番号",
    "種別ID",
    "医療機関名称",
    "都道府県コード",
    "都道府県名",
    "医療機関所在地（郵便番号）",
    "医療機関所在地（住所）",
    "電話番号",
    "FAX番号",
    "病床数",
    "受理届出名称",
    "受理記号",
    "受理番号",
    "算定開始年月日",
    "個別有効開始年月日",
    "更新日",
    "ベッド数"
]
for key, kikan in grouped:
    kikan["ベッド数"] = np.nan

    bed_num = kikan[kikan['備考（見出し）'].str.contains('病床数', na=False)]
    if not bed_num.empty:
        kikan["ベッド数"] = ' '.join(bed_num['備考（データ）'])

    l_data.append(kikan.iloc[0][l_columns])

kikan_df = pd.DataFrame(l_data)
kikan_df.sort_values(by=["医療機関コード", "受理届出名称"], inplace=True)
kikan_df.reset_index(drop=True, inplace=True)
kikan_df

2,医療機関コード,医療機関番号,併設医療機関番号,種別ID,医療機関名称,都道府県コード,都道府県名,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,FAX番号,病床数,受理届出名称,受理記号,受理番号,算定開始年月日,個別有効開始年月日,更新日,ベッド数
0,1810117059,0117059,NaN,1,福井県立病院,18,福井県,910－8526,福井市四ツ井２－８－１,0776-54-5151,0776-57-2945,一般 551\n一般（感染）\n 4\n結核 ...,ハイケアユニット入院医療管理料１,ハイケア１,第4号,令和 4年 8月 1日,NaN,令和 6年10月 1日,8床
1,1810117059,0117059,NaN,1,福井県立病院,18,福井県,910－8526,福井市四ツ井２－８－１,0776-54-5151,0776-57-2945,一般 551\n一般（感染）\n 4\n結核 ...,救命救急入院料１,救１,第2号,令和 6年 4月 1日,NaN,令和 6年10月 1日,16床
2,1810117059,0117059,NaN,1,福井県立病院,18,福井県,910－8526,福井市四ツ井２－８－１,0776-54-5151,0776-57-2945,一般 551\n一般（感染）\n 4\n結核 ...,特定集中治療室管理料６,集６,第1号,令和 6年 6月 1日,NaN,令和 6年10月 1日,NaN
3,1810117117,0117117,NaN,1,福井赤十字病院,18,福井県,918－8501,福井市月見２－４－１,0776-36-3630,0776-36-4133,一般 520\n結核 5\n一般（感染）\n 感染 ...,特定集中治療室管理料６,集６,第2号,令和 6年 8月 1日,NaN,令和 6年10月 1日,NaN
4,1810117117,0117117,NaN,1,福井赤十字病院,18,福井県,918－8501,福井市月見２－４－１,0776-36-3630,0776-36-4133,一般 520\n結核 5\n一般（感染）\n 感染 ...,脳卒中ケアユニット入院医療管理料,脳卒中ケア,第2号,令和 4年 4月 1日,NaN,令和 6年10月 1日,12床
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,3018210124,8210124,8230128,1,日本赤十字社 和歌山医療センター,30,和歌山県,640－8269,和歌山市小松原通４丁目２０番地,073-422-4171,073-426-1168,一般 692\n一般（感染）\n 8,特定集中治療室管理料２,集２,第5号,令和 5年 4月 1日,NaN,令和 6年10月 1日,NaN
399,3018210595,8210595,8230599,1,紀南病院,30,和歌山県,646－8588,田辺市新庄町４６番地７０,0739-22-5000,0739-26-0925,一般 352\n一般（感染）\n 4,特定集中治療室管理料５,集５,第3号,令和 6年 8月 1日,NaN,令和 6年10月 1日,NaN
400,3018410377,8410377,8430371,1,独立行政法人国立病院機構南和歌山医療センター,30,和歌山県,646－0015,田辺市たきない町２７－１,0739-26-7050,0739-24-2055,一般 316,ハイケアユニット入院医療管理料１,ハイケア１,第12号,令和 4年 8月 1日,NaN,令和 6年10月 1日,4床
401,3018410377,8410377,8430371,1,独立行政法人国立病院機構南和歌山医療センター,30,和歌山県,646－0015,田辺市たきない町２７－１,0739-26-7050,0739-24-2055,一般 316,救命救急入院料１,救１,第12号,令和 4年 8月 1日,NaN,令和 6年10月 1日,16床


In [ ]:
kikan_df.to_excel('kikan_df.xlsx', index=False)